# K-Vizinhos Mais Próximos Classificador

## Introdução

Este notebook visa fazer a otimização de **hiperparâmetros**, de estratégia de **normalização**, de **seleção de atributos** e de **redução de dimensionalidade**. A seleção de atributos é realizada por *Sequential Feature Selection* (**SFS**, ou Seleção Sequencial de Atributos em português) por remoção de atributo, a otimização testa a ausência ou não dessa estratégia. A redução de dimensionalidade é realizada por *Principal Component Analysis* (**PCA**, ou Análise de Componenetes Principais em português), é testada sua ausência também. Para normalização, são testadas a ausência, ou a normalização **padrão** ou a pelos **mínimos e máximos**. A otimização é feita usando o módulo **optuna**.

### Modelo

O modelo otimizado é um K-nn classificador do scikit-learn [2]. Os hiperparâmetros otimizados são:

- `n_neighbors`: número de vizinhos. Testados no **intervalo de 2 até 80**, que é um pouco mais de 10% dos 782 registros de tsunami;
    
- `weights`: pesos no cálculo da moda entre os vizinhos mais próximos. Testado entre **'uniform'**, que dá o mesmo peso para todos os vizinhos, e **'distance'**, que dá mais peso aos vizinhos mais próximos;
    
- `p`: potência minkowski. Testado no **intervalo entre 1 e 2**, sendo 1 a distância Manhattan e 2 a distância Euclidiana.

## Carrega os dados

In [1]:
from pandas import read_csv

# Dados de treino
X_treino = read_csv('../Dados/dados_tratados/X_treino.csv')
y_treino = read_csv('../Dados/dados_tratados/y_treino.csv').values.ravel()

## Define algumas variáveis úteis

In [2]:
from sklearn.metrics import fbeta_score, make_scorer

SEMENTE_ALEATORIA = 9 # Random state

f2_score = make_scorer(fbeta_score, beta=2) # Métrica de avaliação de desempenho
num_max_atributos = len(X_treino.columns) - 1 # Para SFS

X_treino = X_treino.values

## Definição de funções para o optuna

O código abaixo define a função que cria uma instância do modelo. É nela que o pipeline é montado de acordo com as sugestões definidas pelo optuna.

In [3]:
from sklearn.neighbors import KNeighborsClassifier # algoritmo de aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalização
from sklearn.feature_selection import SequentialFeatureSelector # seleção de atributos
from sklearn.decomposition import PCA # redução de dimensionalidade
from sklearn.pipeline import make_pipeline

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """

    # trial.suggest_ do optuna
    
    # Hiperparâmetros do algoritmo
    parametros = {
        "n_neighbors": trial.suggest_int("vizinhos", 2, 80),
        "weights": trial.suggest_categorical("pesos", ["uniform", "distance"]), 
        "p": trial.suggest_float("p_minkowski", 1, 2), 
        "n_jobs": -1,
    }

    # Estratégias de tratamento de dados
    normalizacao = trial.suggest_categorical("normalizacao", ["none", "standard", "minmax"])
    usar_sfs = trial.suggest_categorical("SFS", [True, False])
    usar_pca = trial.suggest_categorical("PCA", [True, False])

    # Hiperparâmetros de SFS e PCA, respectivamente
    num_atributos = trial.suggest_int("num_atributos", 2, num_max_atributos) # para SFS
    num_dimensoes = trial.suggest_int("num_dimensoes", 2, num_atributos) # para PCA, depende do num_atributos do SFS
        
    # Monta o pipeline    
    
    steps = []

    # Normalização
    if normalizacao == "standard":
        steps.append(("normalizador", StandardScaler()))
    elif normalizacao == "minmax":
        steps.append(("normalizador", MinMaxScaler()))

    # SFS
    if usar_sfs:
        steps.append(("sfs", SequentialFeatureSelector(
            KNeighborsClassifier(**parametros), 
            n_features_to_select=num_atributos,
            direction="backward",
            scoring=f2_score,
            cv=5,
            n_jobs=1,
        )))

    # PCA
    if usar_pca:
        steps.append(("pca", PCA(n_components=num_dimensoes, random_state=SEMENTE_ALEATORIA)))

    steps.append(("modelo", KNeighborsClassifier(**parametros)))


    # Faz pipeline
    modelo = make_pipeline(*[s[1] for s in steps])

    return modelo

O código abaixo define a função que estima o desempenho do modelo de cada trial. Essa é a função que terá valor de retorno maximizado pelo optuna.

In [4]:
from sklearn.model_selection import cross_val_score
import numpy as np

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna, função cujo retorno será maximizado.

    Faz validação cruzada estratificada com métrica F2.

    Args:
        - trial: objeto tipo Trial do optuna;
        - x: ndarray. dataset de treino com atributos;
        - y: ndaray. dataset de treino com target;
        - num_folds: int. Número de folds da validação cruzada.
    
    Returns:
        - média dos resultados da validação cruzada.
    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring=f2_score,
        cv=num_folds,
    )

    return np.mean(metricas)

## Otimização com optuna

O código abaixo cria ou carrega um 'estudo' a depender da existência dele. O 'estudo' contém informações sobre cada trial. A variável objeto_de_estudo acessa esse 'estudo'. 

In [5]:
from optuna import create_study

NOME_DO_ESTUDO = "Knn_optuna"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 21:24:36,084] A new study created in RDB with name: Knn_optuna


O código abaixo faz a otimização com optuna.

In [6]:
# Parâmetros
NUM_FOLDS = 10 # Número de folds da validação cruzada
NUM_TENTATIVAS = 200 # Quantidade de trials

def funcao_objetivo_parcial(trial):
    '''Função objetivo no formato exigido pelo optuna.'''
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Faz a otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 21:24:36,753] Trial 0 finished with value: 0.8183361795048907 and parameters: {'vizinhos': 35, 'pesos': 'distance', 'p_minkowski': 1.103430415113743, 'normalizacao': 'none', 'SFS': False, 'PCA': True, 'num_atributos': 4, 'num_dimensoes': 3}. Best is trial 0 with value: 0.8183361795048907.
[I 2025-11-03 21:25:38,826] Trial 1 finished with value: 0.8657536534693504 and parameters: {'vizinhos': 26, 'pesos': 'uniform', 'p_minkowski': 1.9181908292224263, 'normalizacao': 'minmax', 'SFS': True, 'PCA': True, 'num_atributos': 2, 'num_dimensoes': 2}. Best is trial 1 with value: 0.8657536534693504.
[I 2025-11-03 21:26:28,188] Trial 2 finished with value: 0.851004237615102 and parameters: {'vizinhos': 23, 'pesos': 'distance', 'p_minkowski': 1.2541413383607345, 'normalizacao': 'standard', 'SFS': True, 'PCA': False, 'num_atributos': 5, 'num_dimensoes': 3}. Best is trial 1 with value: 0.8657536534693504.
[I 2025-11-03 21:27:18,706] Trial 3 finished with value: 0.8438585790426127 and par

## DataFrame dos trials

O código abaixo mostra o DataFrame contruído pelo objeto de estudo do optuna contendo todos os parâmetros de cada trial.

In [7]:
df_analysis = objeto_de_estudo.trials_dataframe()

df_analysis

,number,value,datetime_start,datetime_complete,duration,params_PCA,params_SFS,params_normalizacao,params_num_atributos,params_num_dimensoes,params_p_minkowski,params_pesos,params_vizinhos,state
0,0,0.818336,2025-11-03 21:24:36.117971,2025-11-03 21:24:36.719692,0 days 00:00:00.601721,True,False,none,4,3,1.103430,distance,35,COMPLETE
1,1,0.865754,2025-11-03 21:24:36.769900,2025-11-03 21:25:38.768317,0 days 00:01:01.998417,True,True,minmax,2,2,1.918191,uniform,26,COMPLETE
2,2,0.851004,2025-11-03 21:25:38.845130,2025-11-03 21:26:28.158967,0 days 00:00:49.313837,False,True,standard,5,3,1.254141,distance,23,COMPLETE
3,3,0.843859,2025-11-03 21:26:28.209437,2025-11-03 21:27:18.616490,0 days 00:00:50.407053,False,True,none,3,3,1.747743,distance,26,COMPLETE
4,4,0.828304,2025-11-03 21:27:18.722764,2025-11-03 21:28:09.333619,0 days 00:00:50.610855,False,True,standard,5,5,1.968755,distance,44,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.887095,2025-11-03 23:52:29.015281,2025-11-03 23:53:19.035263,0 days 00:00:50.019982,True,True,none,4,3,1.221676,uniform,63,COMPLETE
196,196,0.873206,2025-11-03 23:53:19.089622,2025-11-03 23:54:24.054230,0 days 00:01:04.964608,True,True,minmax,3,3,1.109423,uniform,66,COMPLETE
197,197,0.835335,2025-11-03 23:54:24.111057,2025-11-03 23:55:30.562445,0 days 00:01:06.451388,True,True,standard,3,3,1.485471,uniform,68,COMPLETE
198,198,0.881690,2025-11-03 23:55:30.615386,2025-11-03 23:56:17.457247,0 days 00:00:46.841861,True,True,none,4,3,1.571421,uniform,58,COMPLETE


## Melhor modelo

O código abaixo mostra o melhor modelo encontrado pelo optuna e seus parâmetros. Note que os parâmetros 'num_atributos' e 'num_dimensoes' só influenciam caso 'SFS' e 'PCA' sejam True respectivamente, pois são seus respectivos hiperparâmetros.

In [8]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 20
Parâmetros do melhor trial: {'vizinhos': 61, 'pesos': 'uniform', 'p_minkowski': 1.2614559083830308, 'normalizacao': 'none', 'SFS': True, 'PCA': True, 'num_atributos': 5, 'num_dimensoes': 4}


Este notebook se encerra aqui. O modelo será usado no notebook chamado 'Resultados_Discussão'.

## Referências

[1] Documentação do optuna. Acesso em: 31/10/2025. Disponível em: https://optuna.readthedocs.io/en/stable/ 

[2] Página da Regressão Logística da documentação do scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

[3] Sperat, Walter. *Using optuna with sklearn the right way*. Acesso em 31/10/2025. Disponível em: https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451